# Refine Company Name 
## 1. data from the below
- "conn = pymysql.connect(host='133.186.143.151', user='eumsread', password='eumsread00!q', db='eums-shared', port=13306,charset='utf8')"

In [1]:
import pandas as pd

data =pd.read_pickle('hana_march.pick')
print(data.shape)
data.head(1)


(9962239, 11)


,EPOPCON_ID,COMPANY_ID,CARD_NAME,CNT_PAY,SUM_PAY,ADDR,CATE,CATE1,CO_NAME,LATITUDE1,LONGITUDE1
0,15a0e888b71e42719f785db8b,27879,기업BC,2,12000.0,경기도 수원시 장안구 송죽동 453-29번지,마트,편의점,CU 수원경기일보점,37.4960912,127.0183798


## PreProcessing 부족한 부분 : cardname, addr, co_name

In [17]:
# p = pd.read_pickle('cardname_clean.pick')

def cleanData(sentence):
    import re
    pattern = re.compile(r'\(.{0,4}\)|RE\)|\<현장즉시결제\>|\[|\s|카드')
    sentence = re.sub(pattern,'',sentence)
    return sentence # return 안쓰면 none값 뜬다.
    
p = p.assign( CARD_NAME = p['CARD_NAME'].apply(cleanData) )
# p.to_pickle('cardname_clean.pick')
p.CARD_NAME.unique()

In [70]:
p = pd.read_pickle('company.pick')

def cleanChar(sentence):
    import re
    pattern = re.compile(r'\s{0,2}\&amp\;\s{0,2}')
    sentence = re.sub(pattern,'@',sentence)
    pattern = re.compile(r'\s{0,2}\&.{2,4}\;\s{0,2}|\<|\>|\'|\"')
    sentence = re.sub(pattern,'',sentence)
    pattern = re.compile(r'\&')
    sentence = re.sub(pattern,'@',sentence)
    return sentence

ts = p[p['ADDR'].str.contains('&')]
# ts = p[p['CO_NAME'].str.contains('\&;')]

# ts[['ADDR','CO_NAME']]=ts[['ADDR','CO_NAME']].applymap(cleanChar)
# p[['ADDR','CO_NAME']]=p[['ADDR','CO_NAME']].applymap(cleanChar)
 # ts=ts.assign(ADDR,CO_NAME=ts[['ADDR','CO_NAME']].apply(cleanChar))

print(ts.head(2))
# print(p.head(2))
# p['CO_NAME']

# p.to_pickle('company.pick')
print(p.shape)

Empty DataFrame
Columns: [ADDR, CATE, CATE1, COMPANY_ID, CO_NAME]
Index: []
(908520, 5)


In [71]:
refined = refineHana(p)
print(refined.shape)
refined[refined['CO_NAME'].str.contains('&')]

total:  (908520, 5)
nonRestroom:  (908459, 5)
Restroom:  (61, 5)
public:  (25, 5)
after public:  (25, 5)
nonPublic:  (36, 5)
Restroon:  (61, 5)
(908520, 5)


,ADDR,CATE,CATE1,COMPANY_ID,CO_NAME


## 2. Explore 

1. & 
2. 화장실 : 공공 / 비공공별 카테고리

In [ ]:
# & -> 앤
#20509
test = data[data['CO_NAME'].str.contains('&')]
print(test.shape)
test.head()

In [10]:
# 화장실 -> '' , 공공 / not 공공

test = data[data['CO_NAME'].str.contains('화장실')]
test_public = test[test['CATE']=='공공']
test_nonPublic = test[test['CATE']!='공공']

print(test.shape)
print(test_public.shape)
print(test_nonPublic.shape)
# test_nonPublic.head()

(2451, 11)
(814, 11)
(1637, 11)


In [11]:
# 공공시설 + 공단
test_public[test_public['CO_NAME'].str.contains('공단')].shape #509

# 공공시절 - 공단
test_public[~test_public['CO_NAME'].str.contains('공단')].shape #305
## 1)
test_public[~test_public['CO_NAME'].str.contains('공단')&test_public['CO_NAME'].str.contains('주유소|충전소|가스')].shape # 263
## 2)
test_public[~test_public['CO_NAME'].str.contains('공단')&~test_public['CO_NAME'].str.contains('주유소|충전소|가스')].shape #42

### 1)-1 교통>주유소
test_public[test_public['CO_NAME'].str.contains('주유소')].shape #180

### 1)-2 교통>충전소
test_public[test_public['CO_NAME'].str.contains('충전소|가스')].shape #78

### 2)-1 의료/건강>병원, 교육>수련원, 공공>사회복지시설
test_public[~test_public['CO_NAME'].str.contains('공단')&test_public['CO_NAME'].str.contains('병원|수련관|복지관')].shape #42

test_public[test_public['CO_NAME'].str.contains('공단')].CO_NAME.unique()

array(['마포구시설관리공단 개방화장실', '구로구시설관리공단 개방화장실'], dtype=object)

## 3. Function testing

### assign
 - 복수 카테고리 변경 / apply 사용시, assign사용
 - 단일 카테고리 변경 시, column에 바로 삽입(assign 미사용)
 
### contains/replace
 - xml 특수기호(<,>,&,',") 제거 <br/>
 : escape기호 사용여부 상관없이 동일결과

In [13]:
A = test_public[test_public['CO_NAME'].str.contains('주유소')]
A = A.assign(CATE = '교통', CATE1 = '주유소')
A.head(1)

# test_public = test_public[test_public['CO_NAME'].str.contains('주유소')].assign(CATE = '교통', CATE1 = '주유소')
# test_public.head()

,EPOPCON_ID,COMPANY_ID,CARD_NAME,CNT_PAY,SUM_PAY,ADDR,CATE,CATE1,CO_NAME,LATITUDE1,LONGITUDE1
121051,15a38df7071e42756fbe6cd42,5860528,현대카드,3,277000.0,서울 동대문구 장안동 466-8,교통,주유소,SK스피드주유소 개방화장실,37.5650989,127.0692033


In [24]:
B = data[data['CO_NAME'].str.contains("'")]
B.head(1)

,EPOPCON_ID,COMPANY_ID,CARD_NAME,CNT_PAY,SUM_PAY,ADDR,CATE,CATE1,CO_NAME,LATITUDE1,LONGITUDE1
2143,15a38abd271e42756fbe69dae,7772765,신한카드,1,18000.0,광주광역시 광산구 송정동 975-9,카페/디저트,베이커리,쑥's 초코파이,37.5071160,127.0561219


In [29]:
B = data[data['CO_NAME'].str.contains('\'|\"')]
B.head(1)

,EPOPCON_ID,COMPANY_ID,CARD_NAME,CNT_PAY,SUM_PAY,ADDR,CATE,CATE1,CO_NAME,LATITUDE1,LONGITUDE1
2143,15a38abd271e42756fbe69dae,7772765,신한카드,1,18000.0,광주광역시 광산구 송정동 975-9,카페/디저트,베이커리,쑥's 초코파이,37.5071160,127.0561219


In [26]:
B.CO_NAME = data['CO_NAME'].str.replace("'",'')
B.head(1)

C:\Users\epopc\Anaconda3\lib\site-packages\pandas\core\generic.py:3110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


,EPOPCON_ID,COMPANY_ID,CARD_NAME,CNT_PAY,SUM_PAY,ADDR,CATE,CATE1,CO_NAME,LATITUDE1,LONGITUDE1
2143,15a38abd271e42756fbe69dae,7772765,신한카드,1,18000.0,광주광역시 광산구 송정동 975-9,카페/디저트,베이커리,쑥s 초코파이,37.5071160,127.0561219


In [27]:
B.CO_NAME = data['CO_NAME'].str.replace("\'",'')
B.head(1)

C:\Users\epopc\Anaconda3\lib\site-packages\pandas\core\generic.py:3110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


,EPOPCON_ID,COMPANY_ID,CARD_NAME,CNT_PAY,SUM_PAY,ADDR,CATE,CATE1,CO_NAME,LATITUDE1,LONGITUDE1
2143,15a38abd271e42756fbe69dae,7772765,신한카드,1,18000.0,광주광역시 광산구 송정동 975-9,카페/디저트,베이커리,쑥s 초코파이,37.5071160,127.0561219


# Apply Refining function

In [73]:
def refineHana(data):
    #특수기호 제거
#     data.CO_NAME=data['CO_NAME'].str.replace('\&','앤') #|₩&
#     data.CO_NAME=data['CO_NAME'].str.replace('\<|\>|\"|\'','')
    print('total: ', data.shape)
    
    #화장실제거
    test0 = data[~data['CO_NAME'].str.contains('화장실')]
    print('nonRestroom: ',test0.shape)
    test = data[data['CO_NAME'].str.contains('화장실')]  
    print('Restroom: ',test.shape)
    
    #1
    test_public = test[test['CATE']=='공공']  
    
    print('public: ',test_public.shape)    
    etc = test_public[~test_public['CO_NAME'].str.contains('주유소|충전소|가스|병원|수련관|복지관')]
    a = test_public[test_public['CO_NAME'].str.contains('주유소')].assign(CATE = '교통', CATE1 = '주유소')
    b = test_public[test_public['CO_NAME'].str.contains('충전소|가스')].assign(CATE = '교통', CATE1 = '충전소')
    c = test_public[test_public['CO_NAME'].str.contains('병원')].assign(CATE = '의료/건강', CATE1 = '병원')
    d = test_public[test_public['CO_NAME'].str.contains('수련관')].assign(CATE = '교육', CATE1 = '수련원')
    e = test_public[test_public['CO_NAME'].str.contains('복지관')].assign(CATE = '공공', CATE1 = '사회복지시설')
    test_public = etc.append([a,b,c,d,e])
    
    print('after public: ',test_public.shape)    
    test_public=test_public.assign(CO_NAME=test_public['CO_NAME'].str.replace('공중화장실','')) #460
    test_public=test_public.assign(CO_NAME=test_public['CO_NAME'].str.replace('개방화장실',''))#1157
    test_public=test_public.assign(CO_NAME=test_public['CO_NAME'].str.replace('화장실',''))#20 - 충전소 화장실, 협동조합 화장실
    # A=A.assign(CO_NAME=A['CO_NAME'].str.replace('공중화장실',''))
    
    #2
    test_nonPublic = test[test['CATE']!='공공']
    print('nonPublic: ',test_nonPublic.shape)
    test_nonPublic=test_nonPublic.assign(CO_NAME=test_nonPublic['CO_NAME'].str.replace('공중화장실','')) #460
    test_nonPublic=test_nonPublic.assign(CO_NAME=test_nonPublic['CO_NAME'].str.replace('개방화장실',''))#1157
    test_nonPublic=test_nonPublic.assign(CO_NAME=test_nonPublic['CO_NAME'].str.replace('화장실',''))#20 - 충전소 화장실, 협동조합 화장실
    # test_nonPublic.CO_NAME=test_nonPublic['CO_NAME'].str.replace('공중화장실','') #460
  
    
    #3
    temp = test_public.append(test_nonPublic)
    print('Restroon: ',temp.shape)
    
#     temp.append(test0).sort_values(by=['EPOPCON_ID','COMPANY_ID','CARD_NAME'], axis=0)
    print(temp.append(test0).shape)
    return  temp.append(test0)


# refined= refineHana(data)
# print(refined[refined["CO_NAME"].str.contains("&")])
# refined.head(1)

In [76]:
refined = refineHana(p)
refined[refined["ADDR"].str.contains("&")]
print(refined[refined["CO_NAME"].str.contains("화장실")])

refined.to_pickle('company.pick')

total:  (908520, 5)
nonRestroom:  (908459, 5)
Restroom:  (61, 5)
public:  (25, 5)
after public:  (25, 5)
nonPublic:  (36, 5)
Restroon:  (61, 5)
(908520, 5)
Empty DataFrame
Columns: [ADDR, CATE, CATE1, COMPANY_ID, CO_NAME]
Index: []


In [129]:
refined.head()

,EPOPCON_ID,COMPANY_ID,CARD_NAME,CNT_PAY,SUM_PAY,ADDR,CATE,CATE1,CO_NAME,LATITUDE1,LONGITUDE1
14085,15a38b7406259ad0a9e317ede,322764,농협카드,1,400.0,서울특별시 마포구 성산동 533-1,공공,공공기관,마포구시설관리공단,37.5667030,126.9783130
47531,15a38c81fe946d9170e4ac4e5,322764,롯데카드,3,10200.0,서울특별시 마포구 성산동 533-1,공공,공공기관,마포구시설관리공단,37.5728003,126.9647050
64841,15a38cf376259ad0a9e3192f3,322764,삼성카드,1,4700.0,서울특별시 마포구 성산동 533-1,공공,공공기관,마포구시설관리공단,37.5113723,127.0628389
91644,15a38d7606259ad0a9e319db7,322764,현대카드,2,4400.0,서울특별시 마포구 성산동 533-1,공공,공공기관,마포구시설관리공단,37.5311155,126.9248774
115238,15a38dd906259ad0a9e31a798,5764271,KB국민카드,2,7800.0,서울 구로구 구로동 741-27,공공,공공기관,구로구시설관리공단,37.5667030,126.9783130


In [31]:
#https://stackoverflow.com/questions/18574108/how-do-convert-a-pandas-dataframe-to-xml

def to_xml(df, filename=None, mode='w'):
    bigxml=['<?xml version="1.0"?>','<resultset xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance">']
    def row_to_xml(row):
        xml = [' <row>']
        for i, col_name in enumerate(row.index):
            xml.append('  <field name="{0}">{1}</field>'.format(col_name, row.iloc[i]))
        xml.append(' </row>')
        return '\n'.join(xml)
    resultset = '\n'.join(df.apply(row_to_xml, axis=1))
    bigxml.append(resultset)
    bigxml.append('</resultset>')
    res = '\n'.join(bigxml)
    
    if filename is None:
        return res
    with open(filename, mode, encoding='utf8') as f:
        f.write(res)

pd.DataFrame.to_xml = to_xml

#tmp.to_xml('test.csv')

p=refined
p.iloc[:2500000,].to_xml('hana_march1.xml')
p.iloc[2500000:5000000,].to_xml('hana_march2.xml')
p.iloc[5000000:7500000,].to_xml('hana_march3.xml')
p.iloc[7500000:9962239,].to_xml('hana_march4.xml')

In [32]:
p.head()

,EPOPCON_ID,COMPANY_ID,CARD_NAME,CNT_PAY,SUM_PAY,ADDR,CATE,CATE1,CO_NAME,LATITUDE1,LONGITUDE1
0,15a0e888b71e42719f785db8b,27879,기업BC,2,12000.0,경기도 수원시 장안구 송죽동 453-29번지,마트,편의점,CU 수원경기일보점,37.4960912,127.0183798
1,15a0e888b71e42719f785db8b,46177,기업BC,2,99300.0,경기 성남시 분당구 정자동 178-1 NAVER 그린팩토리 1층,온라인쇼핑,온라인쇼핑,네이버,37.4960912,127.0183798
2,15a0e888b71e42719f785db8b,207811,기업BC,1,2000.0,서울 서초구 서초동 1690-1 삼풍프라자 지하1층,생활,생활용품,다이소 교대역점,37.4960912,127.0183798
3,15a0e888b71e42719f785db8b,255730,기업BC,1,6000.0,서울 서초구 서초동 1669-6 동진빌딩 1층,외식,한식,돌배기집 교대역점,37.4960912,127.0183798
4,15a0e888b71e42719f785db8b,331119,기업BC,1,7500.0,서울 서초구 서초동 1674-5,외식,패스트푸드,맥도날드 서울교대점,37.4960912,127.0183798


In [36]:
print(p[p["CO_NAME"].str.contains("'")])

Empty DataFrame
Columns: [EPOPCON_ID, COMPANY_ID, CARD_NAME, CNT_PAY, SUM_PAY, ADDR, CATE, CATE1, CO_NAME, LATITUDE1, LONGITUDE1]
Index: []
